#  Submit GW for molecules

In [ ]:
# General imports.
import nglview
import numpy as np
import ipywidgets as ipw
from IPython.display import clear_output
import itertools

from traitlets import dlink

# AiiDA imports.
%aiida
from aiida.orm import Dict as Dict
from aiida.orm import SinglefileData, Node, StructureData

# Workchains.
from aiida.plugins import WorkflowFactory



# AiiDA lab imports.
from aiidalab_widgets_base.utils import string_range_to_list, list_to_string_range
from aiidalab_widgets_base import CodeDropdown, StructureManagerWidget, BasicStructureEditor
from aiidalab_widgets_base import StructureBrowserWidget, StructureUploadWidget, SubmitButtonWidget, SmilesWidget

# Custom imports.
from apps.surfaces.widgets.create_xyz_input_files import make_geom_file
from apps.surfaces.widgets.ANALYZE_structure import StructureAnalyzer
from apps.surfaces.widgets.build_slab import BuildSlab
from apps.surfaces.widgets.cp2k2dict import CP2K2DICT
from apps.surfaces.widgets.inputs import InputDetails
from apps.surfaces.widgets.empa_viewer import EmpaStructureViewer
from apps.surfaces.widgets.import_cdxml import CdxmlUpload2GnrWidget

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
Cp2kAdsorbedGwIcWorkChain = WorkflowFactory('nanotech_empa.cp2k.ads_gw_ic')

In [ ]:
# Structure selector.
#build_slab = BuildSlab(title='build slab')
empa_viewer = EmpaStructureViewer()
#dlink((empa_viewer, 'details'), (build_slab, 'details'))
#dlink((empa_viewer, 'structure'), (build_slab, 'molecule'))

structure_selector = StructureManagerWidget(
    viewer=empa_viewer,
    importers=[
        StructureBrowserWidget(title="AiiDA database"),
        StructureUploadWidget(title="Import from computer"),
        SmilesWidget(title="From SMILES"),
        CdxmlUpload2GnrWidget(title="CDXML"),
    ],
#    editors = [ #If we enable, check the observe on structure..
#        BasicStructureEditor(title="Edit structure") #, build_slab 
#    ],
    storable=False, node_class='StructureData')
display(structure_selector)

# Code.
computer_code_dropdown = CodeDropdown(input_plugin='cp2k')
#input_details = InputDetails()

In [ ]:
# all widgets

In [ ]:
#gw_type
gw_type = ipw.Dropdown(
    options=['gpw_std', 'gapw_std', 'gapw_hq'],
    value='gpw_std',
    description='Protocol:',
    disabled=False,
)


#Multiplicity
multiplicity = ipw.IntText(
    value=1,
    description='Multiplicity',
    disabled=False
)
#UKS
uks = ipw.Checkbox(
    value=False,
    description='UKS',
    disabled=False,
    indent=False
)

#spin guess
spins_up = ipw.Text(
    value='',
    placeholder='1 2 10..13',
    description='Spins U:',
    disabled=False
)

spins_down = ipw.Text(
    value='',
    placeholder='3 4 14..17',
    description='Spins D:',
    disabled=False
)

#geometry mode
geo_mode = ipw.Dropdown(
    options=['ads_geo', 'gas_opt'],
    value='ads_geo',
    description='Geo mode:',
    disabled=False,
)

#geometry mode
ads_height = ipw.FloatText(
    value=0.0,
    description='adsorption height in $\AA$:',
    disabled=False
)
#description
description = ipw.Text(
    value='',
    placeholder='Calculation description',
    description='Description:',
    disabled=False
)
#max nodes
max_nodes = ipw.IntText(
    value=2048,
    description='Max # compute nodes',
    disabled=False
)
#walltime
walltime = ipw.IntText(
    value=3600,
    description='Walltime seconds',
    disabled=False
)

In [ ]:
def get_builder_gw():
    
    builder = Cp2kAdsorbedGwIcWorkChain.get_builder()

    builder.metadata.description = description.value
    builder.code = computer_code_dropdown.selected_code
    
    builder.geometry_mode = Str(geo_mode.value)

   
    #override automatic adsorption height
    if ads_height.value > 0:
        builder.ads_height = Float(ads_height.value)

    ase_geom =  structure_selector.structure        

    # spin guess
    mag_list = [ 0 for t in ase_geom ]
    if uks.value:
        for i in string_range_to_list(spins_up.value)[0]:
            mag_list[i] = 1
        for i in string_range_to_list(spins_down.value)[0]:
            mag_list[i] = -1 
            
        builder.multiplicity = Int(multiplicity.value)
        
    #tags = [abs(i) if i<=0 else i+1 for i in mag_list]   
    #ase_geom.set_tags(tags)
        
    builder.structure = structure_selector.structure_node #StructureData(ase=ase_geom)
    builder.magnetization_per_site = List(list=mag_list)

    builder.protocol = Str(gw_type.value)


    builder.geometry_mode = Str(geo_mode.value)

    debug = 'DEBUG' in description.value
    builder.debug = Bool(debug)
    maxnodes = max_nodes.value
    if debug:
         maxnodes = 1        
    builder.walltime_seconds = Int(walltime.value)
    builder.max_nodes = Int(maxnodes)    
    
    return builder

In [ ]:
def after_submission(_=None):   
    structure_selector.value = None



    
btn_submit_gw = SubmitButtonWidget(Cp2kAdsorbedGwIcWorkChain, 
                                input_dictionary_function=get_builder_gw
                               )
btn_submit_gw.btn_submit.disabled=True

btn_submit_gw.on_submitted(after_submission)

In [ ]:
output = ipw.Output()
def update_all(_=None):
    btn_submit_gw.btn_submit.disabled=False
    #check system
    only_one_molecule = empa_viewer.details['system_type'] == 'SlabXY' 
    only_one_molecule = only_one_molecule or empa_viewer.details['system_type'] == 'Molecule'  
    only_one_molecule = only_one_molecule and  len(empa_viewer.details['all_molecules'])==1    
    msg = 'GW for this system not implemented'
    if only_one_molecule:
        btn_submit_gw.btn_submit.disabled=False  
        msg = ''
    spins_up.value = list_to_string_range(empa_viewer.details['spins_up'])
    spins_down.value = list_to_string_range(empa_viewer.details['spins_down'])
    with output:
        clear_output()
        print(msg)
        if uks.value:
            to_display =  [computer_code_dropdown,gw_type,geo_mode,ads_height,uks,
                           spins_up,spins_down,multiplicity,description,max_nodes,walltime,btn_submit_gw]
        else:
            to_display =  [computer_code_dropdown,gw_type,geo_mode,ads_height,uks,
                           description,max_nodes,walltime,btn_submit_gw]
        display(ipw.VBox(to_display))


structure_selector.observe(update_all, names='structure')
uks.observe(update_all, names='value')

In [ ]:
display(output)